In [1]:
# Import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Import dataset
train_df=pd.read_csv('../data/sales_train_validation.csv')

C:\Users\Yuga\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Yuga\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Yuga\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Yuga\AppData\Local\Continuum

In [2]:
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

In [3]:
train_df=downcast_dtypes(train_df)

In [4]:
train_df = train_df.T
train_df = train_df[6:] # because we need only sales 

In [5]:
sc = MinMaxScaler(feature_range = (0, 1))
train_df = sc.fit_transform(train_df)

In [6]:
timesteps=14
X_train = []
y_train = []
for i in range(timesteps, 1913):
    X_train.append(train_df[i-timesteps:i])
    y_train.append(train_df[i][0:30490]) 

X_train = np.array(X_train, dtype = 'float16')
y_train = np.array(y_train, dtype = 'float16')

X_train.shape,y_train.shape

((1899, 14, 30490), (1899, 30490))

In [7]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(units =64,input_shape = (np.array(X_train).shape[1], np.array(X_train).shape[2])))
model.add(tf.keras.layers.Dense(30490))

model.compile(
  loss='mean_squared_error',
  optimizer=tf.keras.optimizers.Adam(0.001)
)
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                7822080   
_________________________________________________________________
dense (Dense)                (None, 30490)             1981850   
Total params: 9,803,930
Trainable params: 9,803,930
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(X_train, y_train, epochs = 10, batch_size = 10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
1899/1899 [==============================] - 27s 14ms/sample - loss: 0.0141
Epoch 2/10
1899/1899 [==============================] - 23s 12ms/sample - loss: 0.0128
Epoch 3/10
1899/1899 [==============================] - 23s 12ms/sample - loss: 0.0125
Epoch 4/10
1899/1899 [==============================] - 23s 12ms/sample - loss: 0.0123
Epoch 5/10
1899/1899 [==============================] - 24s 13ms/sample - loss: 0.0122
Epoch 6/10
1899/1899 [==============================] - 23s 12ms/sample - loss: 0.0122
Epoch 7/10
1899/1899 [==============================] - 23s 12ms/sample - loss: 0.0121
Epoch 8/10
1899/1899 [==============================] - 23s 12ms/sample - loss: 0.0121
Epoch 9/10
1899/1899 [==============================] - 23s 12ms/sample - loss: 0.0121
Epoch 10/10
1899/1899 [==============================] - 23s 12ms/sample - loss: 0.0121


## Test Data

In [9]:
inputs= train_df[-timesteps:] #14 days in tail
inputs = sc.transform(inputs)

In [10]:
X_test=[]
X_test.append(inputs[0:timesteps])
X_test = np.array(X_test)

In [11]:
predictions=model.predict(X_test)

In [13]:
predictions.shape

(1, 30490)

In [17]:
count = 0
forecast_day = 28
for i in range(forecast_day):
    inputs = train_df[-timesteps:]
    inputs = sc.transform(inputs)
    X_test = [inputs[0:timesteps]]
    X_test = np.array(X_test)
    predictions = model.predict(X_test)
    train_df = np.concatenate((train_df, predictions))
submission = train_df[forecast_day*(-1):]

In [18]:
submission.shape

(28, 30490)